In [21]:
# imports
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats


In [3]:
#  loading in data set
cancer = load_breast_cancer(as_frame=False) # change 'as_frame' to 'True' if you want to see features
X, y = cancer.data, cancer.target
print(f'X:{X} \n\ny:{y}')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)
# print(len(X_train), len(y_test))


X:[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]] 

y:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 

In [4]:
# DecisionTreeClassifier from sklearn

cancer_test = DecisionTreeClassifier(criterion='gini')
cancer_test.fit(X_train, y_train)

pred = cancer_test.predict(X_test)
# print(pred)

In [34]:
# (incomplete) DecisionTreeClassifier from scratch
class DecisionNode:
    def __init__(self, feature_i=None, threshold=None, value=None, true_branch=None, false_branch=None):
        self.feature_i = feature_i
        self.threshold = threshold
        self.value = value
        self.true_branch = true_branch
        self.false_branch = false_branch

def divide_on_feature(X, feature_i, threshold):
    split_func = None
    if isinstance(threshold, int) or isinstance(threshold, float):
        split_func = lambda sample: sample[feature_i] >= threshold
    else:
        split_func = lambda sample: sample[feature_i] == threshold

    X_1 = np.array([sample for sample in X if split_func(sample)])
    X_2 = np.array([sample for sample in X if not split_func(sample)])

    return [X_1, X_2]  # return a list of arrays instead of a numpy array of arrays


class my_DecisionTreeClassifier(object):
    def __init__(self, min_samples_split=2, min_impurity=1e-7, max_depth=float('inf'), loss=None):
        self.root = None
        self.min_samples_split = min_samples_split
        self.min_impurity = min_impurity
        self.max_depth = max_depth
        self._impurity_calculation = None
        self._leaf_value_calculation = None
        self.one_dim = None
        self.loss = loss

    def _calculate_gini(self, y):
        classes = np.unique(y)
        impurity = 1
        for cls in classes:
            p_cls = len(y[y == cls]) / len(y)
            impurity -= p_cls**2
        return impurity
    
    def _calculate_leaf_value(self, y):
        return stats.mode(y)[0]


    def fit(self, X, y, loss=None):
        self.one_dim = len(np.shape(y)) == 1
        self._impurity_calculation = self._calculate_gini
        self._leaf_value_calculation = self._calculate_leaf_value
        self.root = self._build_tree(X, y)
        self.loss=None

    def _build_tree(self, X, y, current_depth=0):
        largest_impurity = 0
        best_criteria = None
        best_sets = None

        if len(np.shape(y)) == 1:
            y = np.expand_dims(y, axis=1)

        Xy = np.concatenate((X, y), axis=1)

        n_samples, n_features = np.shape(X)

        if n_samples >= self.min_samples_split and current_depth <= self.max_depth:
            for feature_i in range(n_features):
                feature_values = np.expand_dims(X[:, feature_i], axis=1)
                unique_values = np.unique(feature_values)

                for threshold in unique_values:
                    Xy1, Xy2 = divide_on_feature(Xy, feature_i, threshold)[0], divide_on_feature(Xy, feature_i, threshold)[1]
                    if len(Xy1) > 0 and len(Xy2) > 0:
                        y1 = Xy1[:, n_features:]
                        y2 = Xy2[:, n_features:]

                        impurity1 = self._calculate_gini(y1)
                        impurity2 = self._calculate_gini(y2)
                        impurity = (len(y1) * impurity1 + len(y2) * impurity2) / len(y)

                        if impurity > largest_impurity:
                            largest_impurity = impurity
                            best_criteria = {"feature_i": feature_i, "threshold": threshold}
                            best_sets = {
                                "leftX": Xy1[:, :n_features],
                                "lefty": Xy1[:, n_features:],
                                "rightX": Xy2[:, :n_features],
                                "righty": Xy2[:, n_features:],
                            }

        if largest_impurity > self.min_impurity:
            true_branch = self._build_tree(best_sets["leftX"], best_sets["lefty"], current_depth + 1)
            false_branch = self._build_tree(best_sets["rightX"], best_sets["righty"], current_depth + 1)
            return DecisionNode(feature_i=best_criteria["feature_i"], threshold=best_criteria["threshold"], true_branch=true_branch, false_branch=false_branch)

        leaf_value = self._leaf_value_calculation(y)

        return DecisionNode(value=leaf_value)

    def predict_value(self, x, tree=None):
        if tree is None:
            tree = self.root

        if tree.value is not None:
            return tree.value

        feature_value = x[tree.feature_i]

        branch = tree.false_branch
        if isinstance(feature_value, int) or isinstance(feature_value, float):
            if feature_value >= tree.threshold:
                branch = tree.true_branch
        elif feature_value == tree.threshold:
            branch = tree.true_branch

        return self.predict_value(x, branch)

    def predict(self, X):
        y_pred = [self.predict_value(sample) for sample in X]
        return y_pred

cancer_test2 = my_DecisionTreeClassifier(max_depth = 10)
cancer_test2.fit(X_train, y_train)
vals = cancer_test2.predict(X_test)

In [37]:
vals2 = []

for i in range(len(vals)):
    vals2.append(int(float(''.join(map(str,vals[i])))))

In [6]:
# kfold cv from sklearn
def sklearn_cv_score(dataset, X, y, k):
    cvs = cross_val_score(estimator = dataset, X=X, y=y, cv=k)
    print(cvs)
    return cvs.mean()

In [7]:
# kfold cv, my own method
def kFoldCV(dataset, X,y,folds=10):
  kf = KFold(n_splits=folds, shuffle=True, random_state=42)
  accuracies=[]

  for train_index, val_index in kf.split(X):
    X_train2, X_val2 = X[train_index], X[val_index]
    y_train2, y_val2 = y[train_index], y[val_index]

    dataset.fit(X_train2, y_train2)
    pred2 = dataset.predict(X_val2)
    
    score = accuracy_score(y_val2, pred2)

    print(f'accuracy per fold, kfold (my own method): {score}')
    accuracies.append(score)

    # print(f'accuracies: {accuracies}')

  avg =  np.mean(accuracies)
  # print(f"kFold CV accuracy: {avg}")
  return avg

In [8]:
# Monte-Carlo cv
def mccv(dataset, X, y, n_iterations=10):
    accuracies = []
    # test = KNN(5)

    for i in range(n_iterations):
        random_indices = np.random.choice(len(X), size=len(X), replace=False)
        X_shuffled = X[random_indices]
        y_shuffled = y[random_indices]
        # print(f'\nX_shuffled: {X_shuffled}, \n\ny_shuffled: {y_shuffled}')

        X_train3, X_test3, y_train3, y_test3 = train_test_split(X_shuffled, y_shuffled, test_size=0.3)
        dataset.fit(X_train3, y_train3)
        y_pred = dataset.predict(X_test3)
        accuracy = np.mean(y_pred == y_test3)
        print(f'accuracy per fold, mccv (my own method): {accuracy}')
        accuracies.append(accuracy)

    return np.mean(accuracies)

In [9]:
# runtime
def runtime(dataset, X, y):
    X_train4, X_test4, y_train4, y_test4 = train_test_split(X, y, test_size=0.3, random_state = 42)
    dataset.fit(X_train4, y_train4)
    start_time = time.time()
    dataset.predict(X_test4)
    end_time = time.time()

    total = end_time - start_time
    return total

In [10]:
# data visualization
def viz(X, y):
    df = pd.DataFrame(X)
    print(f'\n\ndf: {df}')
    df['target'] = y
    sns.pairplot(df, hue='target', palette='Set1')
    plt.show()

In [38]:
# outputs
print(f'( DecisionTreeClassifier from sklearn) ( kfold cv from sklearn ): {sklearn_cv_score(cancer_test, X, y, 10)}\n')

print(f'( DecisionTreeClassifier from sklearn) ( kfold cv, my own method ): {kFoldCV(cancer_test, X, y)}\n')

print(f' ( DecisionTreeClassifier from sklearn ) ( Monte-Carolo cv ): {mccv(cancer_test, X, y)}\n')

print(f' \n( DecisionTreeClassifier from sklearn) ( runtime ): {runtime(cancer_test, X, y)}')

print(f'( DecisionTreeClassifier from scratch ) ( loading in data set ): ', vals2)

# print(f' ( DecisionTreeClassifier from sklearn) ( data visualization ):')
# viz(X, y)

[0.89473684 0.85964912 0.92982456 0.87719298 0.96491228 0.89473684
 0.87719298 0.94736842 0.9122807  0.96428571]
( DecisionTreeClassifier from sklearn) ( kfold cv from sklearn ): 0.9122180451127818

accuracy per fold, kfold (my own method): 0.9122807017543859
accuracy per fold, kfold (my own method): 0.9473684210526315
accuracy per fold, kfold (my own method): 0.9649122807017544
accuracy per fold, kfold (my own method): 0.8947368421052632
accuracy per fold, kfold (my own method): 0.9122807017543859
accuracy per fold, kfold (my own method): 0.8771929824561403
accuracy per fold, kfold (my own method): 0.9473684210526315
accuracy per fold, kfold (my own method): 0.9649122807017544
accuracy per fold, kfold (my own method): 0.9649122807017544
accuracy per fold, kfold (my own method): 0.9464285714285714
( DecisionTreeClassifier from sklearn) ( kfold cv, my own method ): 0.9332393483709274

accuracy per fold, mccv (my own method): 0.935672514619883
accuracy per fold, mccv (my own method): 0.9